Imports

In [1]:
from pre_process import AmazonDatasetPreprocessor, KaggleDatasetPreprocessor, SentenceDatasetPreprocessor
from config import PROCESSED_DATA_PATH
import os
import pandas as pd
from gpt import ChatGPTSession

import sys
sys.path.insert(0, '..')

from credentials import openai_api_key

/Users/abhivellore/LLMSentiment/saenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Preprocess Data

In [ ]:
amazon_processor = AmazonDatasetPreprocessor('Amazon_Fashion_Review_Data.json')
amazon_processor.preprocess()
amazon_processor.to_csv('processed_amazon_data.csv')

# Initialize and process the Sentence data
sentence_processor = SentenceDatasetPreprocessor('Sentences_75Agree.txt')
sentence_processor.preprocess()
sentence_processor.to_csv('processed_sentence_data.csv')

# Initialize and process the first Kaggle dataset
kaggle1_processor = KaggleDatasetPreprocessor('kaggle_train.csv')
kaggle1_processor.preprocess()
kaggle1_processor.to_csv('processed_kaggle_train_data.csv')

# Initialize and process the second Kaggle dataset
kaggle2_processor = KaggleDatasetPreprocessor('kaggle_test.csv')
kaggle2_processor.preprocess()
kaggle2_processor.to_csv('processed_kaggle_test_data.csv')

# Concatenate the preprocessed DataFrames
processed_kaggle1_df = kaggle1_processor.df
processed_kaggle2_df = kaggle2_processor.df
combined_df = pd.concat([processed_kaggle1_df, processed_kaggle2_df], ignore_index=True)

# Save the combined DataFrame to a new CSV file in the processed directory
# Since combined_df is not associated with a processor, we need to handle the path manually
combined_csv_path = os.path.join(PROCESSED_DATA_PATH, 'processed_kaggle_combined_data.csv')
combined_df.to_csv(combined_csv_path, index=False)
print(f"Processed combined Kaggle data saved to {combined_csv_path}")

GPT Requests

In [2]:
input_csv_path = 'test_experiment.csv'
output_csv_path = 'test_result.csv'

In [4]:
sentiment_context = "As a sentiment analysis model, rate the sentiment of the following text from 1 to 5, where 1 is very negative and 5 is very positive. Provide only the number as a response."
    
chat_session = ChatGPTSession(api_key=openai_api_key)
chat_session.set_context(sentiment_context)

# Example test call to check the raw response structure
test_response = chat_session.send_prompt("Test prompt")
print(test_response)


# df = pd.read_csv(input_csv_path)
# df['Sentiment Score'] = df['Text'].apply(lambda x: chat_session.send_prompt(x))
# df.to_csv(output_csv_path, index=False)

{'error': {'message': 'The model `gpt-3.5` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


In [3]:
# Initialize the sentiment analysis session
sentiment_session = SentimentAnalysisSession2(openai_api_key)

# Load the CSV file into a DataFrame
df = pd.read_csv(input_csv_path)

# Process each row to analyze sentiment and append the result
df['Sentiment Score'] = df['Text'].apply(lambda text: sentiment_session.get_sentiment_score(text))

# Save the DataFrame with the sentiment scores back to a new CSV file
df.to_csv(output_csv_path, index=False)

# Display the DataFrame with the sentiment scores
df.head()

Tokens used for this request: 23
Total tokens used so far: 23
Tokens used for this request: 24
Total tokens used so far: 47
Tokens used for this request: 7
Total tokens used so far: 54
Tokens used for this request: 28
Total tokens used so far: 82


,Text,Score,Sentiment Score
0,"Recession hit Veronique Branquinho, she has to...",NaN,Score: 2\n\nThe text conveys a negative sentim...
1,about to go to sleep,NaN,I would rate this as a 4 out of 5. Going to sl...
2,6th pair best thing ever plantar fasciitis res...,NaN,I would rate this text a 2.
3,used inserts years . provide great support .,NaN,I would rate this text as a 4 out of 5. It see...
